In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import geopandas as gpd
import geoplot.crs as gcrs
import geoplot as gplt
from geopy.distance import great_circle
from shapely.geometry import Polygon, LineString, Point, MultiPolygon

In [19]:
# Load the main data frame
df = pd.read_csv('data/df_sf_20240911.csv')

In [20]:
df.head()

,image_id,user_id,sequence_id,image_coord,image_timestamp_unix,image_url,image_angle,chunk_id,image_timestamp,year,month,geoid,bboxid,num_tents,confidence,day,hour,GEOID,geometry
0,327669609883423,105986451641170,tpibOxJS5TYZ8V4crsIX9n,"[-122.49810039997101, 37.708604958036545]",1699723630093,https://scontent-iad3-2.xx.fbcdn.net/m1/v/t6/A...,4.955116,chunk_0,2023-11-11 17:27:10.093,2023,11,60750604002,bbox_2_7,NaN,NaN,11,17,60750604002,"POLYGON ((-122.512733380415 37.73508797666425,..."
1,303401841368941,102905365286641,14jeyr2dygbpaexrxv7bqb,"[-122.49807357788086, 37.708991157486594]",1495412840007,https://scontent-iad3-2.xx.fbcdn.net/m1/v/t6/A...,7.626879,chunk_0,2017-05-22 00:27:20.007,2017,5,60750604002,bbox_2_7,0.0,(),22,0,60750604002,"POLYGON ((-122.512733380415 37.73508797666425,..."
2,329903498499807,104408981801642,v66wj3bf8pakks2pq1ws47,"[-122.49806821346283, 37.70853281066343]",1545518762168,https://scontent-iad3-2.xx.fbcdn.net/m1/v/t6/A...,5.560757,chunk_0,2018-12-22 22:46:02.168,2018,12,60750604002,bbox_2_7,0.0,(),22,22,60750604002,"POLYGON ((-122.512733380415 37.73508797666425,..."
3,274600944394813,104408981801642,52l7p54xlo9r92li3feskq,"[-122.4981701374054, 37.708961449908045]",1531338525997,https://scontent-iad3-2.xx.fbcdn.net/m1/v/t6/A...,183.032623,chunk_0,2018-07-11 19:48:45.997,2018,7,60750604002,bbox_2_7,0.0,(),11,19,60750604002,"POLYGON ((-122.512733380415 37.73508797666425,..."
4,519584729064799,104408981801642,d85uqp4g9lwq0vzt32vao1,"[-122.49844908714294, 37.70762884056478]",1531702119180,https://scontent-iad3-2.xx.fbcdn.net/m1/v/t6/A...,185.561066,chunk_0,2018-07-16 00:48:39.180,2018,7,60750604002,bbox_2_7,0.0,(),16,0,60750604002,"POLYGON ((-122.512733380415 37.73508797666425,..."


In [21]:
# Add 0 to the beginning of GEOID
df['geoid'] = df['geoid'].astype(str)
df['geoid'] = df['geoid'].apply(lambda x: x.zfill(12))

# Drop GEOID
df = df.drop(columns=['GEOID'])

# Sort the data frame
df = df.sort_values(by=['bboxid', 'user_id', 'sequence_id', 'image_timestamp'])

# Reset index
df = df.reset_index(drop=True)

# Make sure 'image_timestamp' is in datetime format
df['image_timestamp'] = pd.to_datetime(df['image_timestamp_unix'], unit='ms')

In [22]:
df.shape

(1852845, 18)

In [23]:
# Load the ground truth data
# Extract image_id info from the 'img_secondary_classifer' folder
joined_files = os.path.join("/Users/wooyongjung/WJ_Projects/HomelessStudy_SanFrancisco_2024/data/img_secondary_classifier/test/yes", "*.jpg")
test_list = glob.glob(joined_files)
joined_files = os.path.join("/Users/wooyongjung/WJ_Projects/HomelessStudy_SanFrancisco_2024/data/img_secondary_classifier/train_and_valid/train/yes", "*.jpg")
train_list = glob.glob(joined_files)
joined_files = os.path.join("/Users/wooyongjung/WJ_Projects/HomelessStudy_SanFrancisco_2024/data/img_secondary_classifier/train_and_valid/val/yes", "*.jpg")
val_list = glob.glob(joined_files)
yes_list = test_list + train_list + val_list

# Extract only the image_id part from the yes_list and no_list
import re
yes_list = [re.search(r"img_\d+", x).group() for x in yes_list]

# Extract only the number from the image_id
yes_list = [re.search(r"\d+", x).group() for x in yes_list]

In [24]:
# Create a ground truth data frame
df_gt = pd.DataFrame({'image_id': yes_list, 'ground_truth': 1})

# Drop duplicates
df_gt = df_gt.drop_duplicates(subset=['image_id'], keep='first')

# Convert dtype of image_id to string
df['image_id'] = df['image_id'].astype(str)

# Merge the ground truth data frame to the main data frame
df = pd.merge(df, df_gt, on='image_id', how='left')

# Fill NaN values with 0
df['ground_truth'] = df['ground_truth'].fillna(0)

In [27]:
# Temporal aspect of the data
# Create a new column 'time_diff'
df_bbox = df.copy()
df_bbox['time_diff'] = df_bbox.groupby(['bboxid', 'sequence_id', str('ground_truth')], as_index=False)['image_timestamp'].diff()

# Fill NaT values with Inf
df_bbox['time_diff'] = df_bbox['time_diff'].fillna(pd.Timedelta(seconds=1e9))

In [29]:
# Sort the data frame
df_bbox = df_bbox.sort_values(by=['bboxid', 'user_id', 'sequence_id', 'image_timestamp'])

# Reset index
df_bbox = df_bbox.reset_index(drop=True)

In [31]:
# Create latitude and longitude columns
df_bbox['latitude'] = df_bbox['image_coord'].apply(lambda x: str(x.split(', ')[1]))
df_bbox['longitude'] = df_bbox['image_coord'].apply(lambda x: str(x.split(', ')[0]))

df_bbox['latitude'] = df_bbox['latitude'].apply(lambda x: float(re.search(r'\d+.\d+', x).group()))
df_bbox['longitude'] = df_bbox['longitude'].apply(lambda x: float(re.search(r'\-\d+.\d+', x).group()))

In [32]:
# Load bbox coordinates
bbox_coord = pd.read_csv('data/df_bbox_99.csv')

# Select the rows with GEOID starting with 06075 (San Francisco County)
bbox_coord['GEOID'] = bbox_coord['GEOID'].astype(str)
bbox_coord['GEOID'] = bbox_coord['GEOID'].apply(lambda x: x.zfill(12)) # Add 0 to the beginning of GEOID
bbox_coord = bbox_coord[bbox_coord['GEOID'].str.startswith('06075')]

# Reset index
bbox_coord.reset_index(inplace=True, drop=True)

# Drop the first column Unnamed: 0
bbox_coord = bbox_coord.drop(['Unnamed: 0'], axis=1)

# Create bboxid column
bbox_coord['bboxid'] = 'bbox_' + bbox_coord.row_num.astype(str) + '_' + bbox_coord.col_num.astype(str)

# Merge the main data frame with the bbox data frame
df_bbox = pd.merge(df_bbox, bbox_coord[['bboxid','swne_edges','center_latlon']], on='bboxid', how='left')

# Create coordinates
df_bbox['swne_edges'] = df_bbox['swne_edges'].apply(lambda x: x[1:-1].split(','))

coords = []
for i in range(len(df_bbox)):
    coord = [(df_bbox['swne_edges'][i][1], df_bbox['swne_edges'][i][0]),
             (df_bbox['swne_edges'][i][3], df_bbox['swne_edges'][i][0]),
             (df_bbox['swne_edges'][i][3], df_bbox['swne_edges'][i][2]),
             (df_bbox['swne_edges'][i][1], df_bbox['swne_edges'][i][2])]
    coords.append(coord)

df_bbox['coords'] = coords

In [35]:
from shapely.geometry import Polygon, LineString, Point, MultiPolygon
# Create a new GeoDataFrame based on coords
# Apply Polygon to coords
df_geo_bbox = df_bbox.copy()
df_geo_bbox['geometry_bbox'] = df_geo_bbox['coords'].apply(Polygon)

# Create a new GeoDataFrame based on coords
df_geo_bbox = gpd.GeoDataFrame(df_geo_bbox, geometry=df_geo_bbox['geometry_bbox'], crs=4326)

In [36]:
# Create a new column 'quarter'
def quarter(row):
    if row['month'] <= 3:
        return 'Q1'
    elif row['month'] <= 6:
        return 'Q2'
    elif row['month'] <= 9:
        return 'Q3'
    else:
        return 'Q4'
    
df_geo_bbox['quarter'] = df_geo_bbox.apply(quarter, axis=1)

In [37]:
df_geo_bbox[df_geo_bbox['year'] == 2024]['month'].value_counts()

month
2    1570
5    1131
4     438
3     369
7     174
8      95
Name: count, dtype: int64

In [38]:
# Remove July and August 2024 data
df_geo_bbox = df_geo_bbox[~((df_geo_bbox['year'] == 2024) & (df_geo_bbox['month'].isin([7, 8])))]

In [39]:
df_geo_bbox['ground_truth'].value_counts()

ground_truth
0.0    1852189
1.0        387
Name: count, dtype: int64

### Merge 311 Dataset

In [40]:
# Load the 311 data
df_311 = pd.read_csv('data/df_311_homeless_sf.csv')

# Convert the 'Opened' column to datetime
df_311['Opened'] = pd.to_datetime(df_311['Opened'])

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_11055/881442386.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_311['Opened'] = pd.to_datetime(df_311['Opened'])


In [41]:
# Filter the data frame based on the 'Opened' data from January 1, 2016 to May 31, 2024
df_311 = df_311[(df_311['Opened'] >= '2016-01-01') & (df_311['Opened'] <= '2024-05-31')]

# Create Year, Month, and Day columns
df_311['Year'] = df_311['Opened'].dt.year
df_311['Month'] = df_311['Opened'].dt.month
df_311['Day'] = df_311['Opened'].dt.day

# Remove Nan values in the 'Point' column
df_311 = df_311[df_311['Point']!= '(0.0, 0.0)']

# Create latitude and longitude columns
df_311['latitude'] = df_311['Point'].apply(lambda x: str(x.split(', ')[0]))
df_311['longitude'] = df_311['Point'].apply(lambda x: str(x.split(', ')[1]))
df_311['latitude'] = df_311['latitude'].apply(lambda x: float(re.search(r'\d+.\d+', x).group()))
df_311['longitude'] = df_311['longitude'].apply(lambda x: float(re.search(r'\-\d+.\d+', x).group()))

In [42]:
df_311.shape

(382630, 12)

In [43]:
# Drop NaN values
df_311 = df_311.dropna(subset=['Status Notes'])
df_311 = df_311.drop(['Unnamed: 0'], axis=1)

# Filter out specific Status Notes
#df_311 = df_311[~df_311['Status Notes'].str.contains('Duplicate')]
df_311 = df_311[~df_311['Status Notes'].str.contains('Unable to Locate')]
df_311 = df_311[~df_311['Status Notes'].str.contains('Case is Invalid')]

In [44]:
# Add Ground-truth column
def ground_truth(row):
    if 'no encampment in area' in row['Status Notes']:
        return 0
    else:
        return 1
    
df_311['ground_truth'] = df_311.apply(ground_truth, axis=1)

In [45]:
df_311['ground_truth'].value_counts()

ground_truth
1    332909
0     25226
Name: count, dtype: int64

In [46]:
df_311.head()

,CaseID,Opened,Category,Point,point_geom,Status Notes,Year,Month,Day,latitude,longitude,ground_truth
0,13380611,2021-01-17 20:56:55,Encampments,"(37.77807304, -122.40828068)",POINT (-122.408280677 37.778073041),Case Resolved - Area was cleaned around the en...,2021,1,17,37.778073,-122.408281,1
1,13390197,2021-01-20 06:50:29,Encampments,"(37.7870072, -122.3959002)",POINT (-122.395900198 37.787007203),Case Transferred - Area was cleaned around the...,2021,1,20,37.787007,-122.395900,1
2,13381074,2021-01-18 07:52:10,Encampments,"(37.78168967, -122.40908995)",POINT (-122.40908995 37.78168967),Case Transferred - Area was cleaned around the...,2021,1,18,37.781690,-122.409090,1
3,13381176,2021-01-18 08:20:54,Encampments,"(37.7837489, -122.4180329)",POINT (-122.4180329 37.7837489),Area Cleared,2021,1,18,37.783749,-122.418033,1
4,13376149,2021-01-16 10:29:00,Encampments,"(37.78515269, -122.39415709)",POINT (-122.39415709 37.78515269),Outreach provided per accordance to CDC Covid-...,2021,1,16,37.785153,-122.394157,1


In [47]:
# Create df_mapi with required columns
df_mapi = df_geo_bbox.copy()
df_mapi = df_mapi[['latitude', 'longitude', 'image_timestamp', 'ground_truth', 'bboxid', 'geometry_bbox', 'center_latlon']]

# Create df_311_mapi with required columns and remove the rows with (0.0, 0.0) coordinates
df_311_mapi = df_311.copy()
df_311_mapi = df_311_mapi[['Opened', 'Category', 'latitude', 'longitude', 'ground_truth']]
df_311_mapi = df_311_mapi.reset_index(drop=True)

# Rename columns
df_mapi.rename(columns={'image_timestamp': 'timestamp'}, inplace=True)
df_311_mapi.rename(columns={'Opened': 'timestamp', 'Category':'category'}, inplace=True)

# Create a new column
df_mapi['source'] = 'Mapillary'
df_mapi['category'] = 'Homeless Tents'
df_311_mapi['source'] = '311'

In [48]:
print(df_mapi.shape)
print(df_311_mapi.shape)

(1852576, 9)
(358135, 6)


In [ ]:
# # Load bbox coordinates
# bbox_coord = pd.read_csv('data/df_bbox_99.csv')

# # Select the rows with GEOID starting with 06075 (San Francisco County)
# bbox_coord['GEOID'] = bbox_coord['GEOID'].astype(str)
# bbox_coord['GEOID'] = bbox_coord['GEOID'].apply(lambda x: x.zfill(12)) # Add 0 to the beginning of GEOID
# bbox_coord = bbox_coord[bbox_coord['GEOID'].str.startswith('06075')]

# # Reset index
# bbox_coord.reset_index(inplace=True, drop=True)

# # Drop the first column Unnamed: 0
# bbox_coord = bbox_coord.drop(['Unnamed: 0'], axis=1)

# # Create bboxid column
# bbox_coord['bboxid'] = 'bbox_' + bbox_coord.row_num.astype(str) + '_' + bbox_coord.col_num.astype(str)

In [13]:
# Create coordinates
bbox_coord['swne_edges'] = bbox_coord['swne_edges'].apply(lambda x: x[1:-1].split(','))

coords = []
for i in range(len(bbox_coord)):
    coord = [(bbox_coord['swne_edges'][i][1], bbox_coord['swne_edges'][i][0]),
             (bbox_coord['swne_edges'][i][3], bbox_coord['swne_edges'][i][0]),
             (bbox_coord['swne_edges'][i][3], bbox_coord['swne_edges'][i][2]),
             (bbox_coord['swne_edges'][i][1], bbox_coord['swne_edges'][i][2])]
    coords.append(coord)

bbox_coord['coords'] = coords

# Apply Polygon to bbox_coord
bbox_coord['geometry_bbox'] = bbox_coord['coords'].apply(Polygon)

# Create a new GeoDataFrame based on coords
bbox_coord = gpd.GeoDataFrame(bbox_coord, geometry=bbox_coord['geometry_bbox'], crs=4326)

In [26]:
bbox_coord.head()

,row_num,col_num,swne_edges,center_latlon,center_lat,center_lon,GEOID,bboxid,coords,geometry_bbox,geometry
0,2,0,"[37.70764501104698, -122.5119109755011, 37.7...","(37.70836648101098, -122.51099334475741)",37.708366,-122.510993,060759901000,bbox_2_0,"[( -122.5119109755011, 37.70764501104698), ( -...",POLYGON ((-122.5119109755011 37.70764501104698...,"POLYGON ((-122.51191 37.70765, -122.51008 37.7..."
1,2,1,"[37.70763743908666, -122.51008524507708, 37....","(37.70835889471916, -122.50916759688505)",37.708359,-122.509168,060759901000,bbox_2_1,"[( -122.51008524507708, 37.70763743908666), ( ...",POLYGON ((-122.51008524507708 37.7076374390866...,"POLYGON ((-122.51009 37.70764, -122.50825 37.7..."
2,2,2,"[37.70762983885778, -122.50825951533793, 37....","(37.70835128015806, -122.50734184969886)",37.708351,-122.507342,060759901000,bbox_2_2,"[( -122.50825951533793, 37.70762983885778), ( ...",POLYGON ((-122.50825951533793 37.7076298388577...,"POLYGON ((-122.50826 37.70763, -122.50642 37.7..."
3,2,3,"[37.70762221036033, -122.50643378628618, 37....","(37.70834363732769, -122.50551610320136)",37.708344,-122.505516,060750604002,bbox_2_3,"[( -122.50643378628618, 37.70762221036033), ( ...",POLYGON ((-122.50643378628618 37.7076222103603...,"POLYGON ((-122.50643 37.70762, -122.5046 37.70..."
4,2,4,"[37.707614553594375, -122.5046080579244, 37....","(37.708335966228084, -122.50369035739516)",37.708336,-122.503690,060750604002,bbox_2_4,"[( -122.5046080579244, 37.707614553594375), ( ...",POLYGON ((-122.5046080579244 37.70761455359437...,"POLYGON ((-122.50461 37.70761, -122.50277 37.7..."


In [51]:
# # Load a geojson file
# df_bboxid = gpd.read_file('data/df_bbox_10142024.geojson')
# df_bboxid['bboxid'] = 'bbox_' + df_bboxid['row_num'].astype(str) + '_' + df_bboxid['col_num'].astype(str)
# df_bboxid = df_bboxid.drop(['row_num', 'col_num'], axis=1)

# # Convert df_bboxid to GeoDataFrame
# df_bboxid = gpd.GeoDataFrame(df_bboxid, geometry=df_bboxid['geometry'], crs=4326)

# Convert df_311_mapi to GeoDataFrame
df_311_mapi = gpd.GeoDataFrame(df_311_mapi, geometry=gpd.points_from_xy(df_311_mapi.longitude, df_311_mapi.latitude), crs=4326)

In [52]:
print(df_311_mapi.shape)
print(bbox_coord.shape)

(358135, 7)
(7439, 11)


In [53]:
# Spatial join
df_311_bbox = gpd.sjoin(df_311_mapi, bbox_coord[['bboxid', 'geometry', 'center_latlon']], how='left', predicate='within')
df_311_bbox = df_311_bbox.drop_duplicates(keep='first')
df_311_bbox = df_311_bbox.reset_index(drop=True)

In [56]:
# Extract only the rows with null bboxid
df_311_null = df_311_bbox[df_311_bbox['bboxid'].isnull()]
df_311_null.drop(['index_right','bboxid'], axis=1, inplace=True)
df_311_null = df_311_null.reset_index(drop=True)

# Drop the null bboxid from df_311_bbox
df_311_bbox = df_311_bbox.dropna(subset=['bboxid'])

# Spatial join using sjoin_nearest for the null bboxid
df_311_null = gpd.sjoin_nearest(df_311_null, bbox_coord[['bboxid', 'geometry', 'center_latlon']], how='left', max_distance=1000)

# Concatenate df_311_bbox and df_311_null
df_311_bbox = pd.concat([df_311_bbox, df_311_null])

# Merge df_311_bbox and bbox_coord to get the geometry_bbox column
bbox_coord_geobbox = bbox_coord.copy()
df_311_bbox = pd.merge(df_311_bbox, bbox_coord_geobbox[['bboxid', 'geometry_bbox','center_latlon']], on='bboxid', how='left')

df_311_bbox.shape


/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_11055/1358374698.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_311_null.drop(['index_right','bboxid'], axis=1, inplace=True)
/Users/wooyongjung/miniconda3/envs/homeless_env/lib/python3.11/site-packages/geopandas/array.py:417: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


(360135, 14)

In [57]:
df_311_mapi.shape

(358135, 7)

In [58]:
# Drop unnecessary columns and rename columns
df_311_bbox = df_311_bbox.drop(['center_latlon_left', 'center_latlon_right', 'center_latlon_y'], axis=1)
df_311_bbox.rename(columns={'center_latlon_x': 'center_latlon'}, inplace=True)

In [60]:
print('Mapillary: ', df_mapi['ground_truth'].value_counts())
print('311 Service Call:', df_311_bbox['ground_truth'].value_counts())

Mapillary:  ground_truth
0.0    1852189
1.0        387
Name: count, dtype: int64
311 Service Call: ground_truth
1    334778
0     25357
Name: count, dtype: int64


In [61]:
print(df_mapi.shape)
print(df_311_bbox.shape)

(1852576, 9)
(360135, 11)


In [62]:
# Concatenate the two data frames
df_combined = pd.concat([df_mapi[['timestamp','bboxid','latitude','longitude','ground_truth','category','source','geometry_bbox', 'center_latlon']], 
                         df_311_bbox[['timestamp','bboxid','latitude','longitude','ground_truth','category','source','geometry_bbox', 'center_latlon']]])

In [63]:
df_combined.shape

(2212711, 9)

In [64]:
df_combined['ground_truth'].value_counts()

ground_truth
0.0    1877546
1.0     335165
Name: count, dtype: int64

In [ ]:
# #df_combined.to_csv('data/df_combined_final_20250116.csv', index=False)
# # Load the combined data frame
# df_combined = pd.read_csv('data/df_combined_final_20250116.csv')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_11055/1335385016.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv('data/df_combined_final_20250116.csv')


In [65]:
df_combined.head(3)

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon
0,2015-08-14 14:45:35.348,bbox_10_14,37.719193,-122.485049,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"
1,2015-08-14 14:45:37.347,bbox_10_14,37.719490,-122.485049,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"
2,2015-08-14 14:45:39.343,bbox_10_14,37.719902,-122.485054,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"


In [66]:
# Filter the data frame for January 1, 2016 to May 31, 2024
df_combined = df_combined[(df_combined['timestamp'] >= '2016-01-01') & (df_combined['timestamp'] <= '2024-05-31')]
df_combined.shape

(2163988, 9)

In [67]:
# Extract year, month, and day
df_combined['year'] = df_combined['timestamp'].dt.year
df_combined['month'] = df_combined['timestamp'].dt.month
df_combined['day'] = df_combined['timestamp'].dt.day

In [ ]:
# df_combined.to_csv('data/df_combined_final_20250116.csv', index=False)
# # # Load the combined data frame
# # df_combined = pd.read_csv('data/df_combined_final_20250116.csv')

In [69]:
# Remove overlapping points
def remove_overlapping_obs(df, diameter):
    """
    Removes the overlapping observations reported within a certain diameter.

    Parameters:
    - df: pandas DataFrame with 'latitude' and 'longitude' columns
    - *args: columns to group by (conditions)
    - diameter: the diameter (in meters) within which points are considered overlapping

    Returns:
    - Filtered DataFrame with non-overlapping points
    """
    # Create a new DataFrame to store non-overlapping points
    filtered_points = []

    for index, row in df.iterrows():
        point = (row['latitude'], row['longitude'])
        
        # Check if the point is within the specified diameter of any previously added point
        if not any(great_circle(point, (p['latitude'], p['longitude'])).meters <= diameter for p in filtered_points):
            filtered_points.append(row)
        
    return pd.DataFrame(filtered_points)


# Remove overlapping observations (within 10 meters)
df_combined2 = df_combined.copy()
df_combined2 = df_combined2.groupby(['bboxid','year','month','day','ground_truth'], as_index=False).apply(remove_overlapping_obs, diameter=10)

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_11055/2223803197.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_combined2 = df_combined2.groupby(['bboxid','year','month','day','ground_truth'], as_index=False).apply(remove_overlapping_obs, diameter=10)


In [70]:
df_combined.head(3)

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon,year,month,day
14,2016-11-09 14:15:57,bbox_10_14,37.720555,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
15,2016-11-09 14:15:58,bbox_10_14,37.720457,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
16,2016-11-09 14:15:59,bbox_10_14,37.720360,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9


In [72]:
print(f'before removing overlaping point: {df_combined.shape}')
print(f'after removing overlaping points: {df_combined2.shape}')

before removing overlaping point: (2163988, 12)
after removing overlaping points: (703796, 12)


In [5]:
#df_combined2.to_csv('data/df_combined_final_20250116_remove_overlap.csv', index=False)
# Load the combined data frame
df_combined2 = pd.read_csv('data/df_combined_final_20250116_remove_overlap.csv')

In [6]:
# Convert df_combined to geo data frame
#df_combined2 = gpd.GeoDataFrame(df_combined2, geometry='geometry_bbox', crs=4326)
df_combined2 = df_combined2.sort_values(by=['bboxid','timestamp'])

# Reset index
df_combined2 = df_combined2.reset_index(drop=True)

# Fill NaN values in 'geometry_bbox' and 'center_latlon' with the first non-null value in each group
df_combined2['geometry_bbox'] = df_combined2.groupby('bboxid')['geometry_bbox'].transform(lambda x: x.ffill().bfill())
df_combined2['center_latlon'] = df_combined2.groupby('bboxid')['center_latlon'].transform(lambda x: x.ffill().bfill())

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/3380555379.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined2['center_latlon'] = df_combined2.groupby('bboxid')['center_latlon'].transform(lambda x: x.ffill().bfill())


In [16]:
# Separate the data frame for groupby
group_col = ['bboxid','year', 'month', 'day', 'ground_truth']
df_combined_grp = df_combined2[group_col].copy()
df_combined_grp = df_combined_grp.groupby(['bboxid','year', 'month', 'day']).sum().reset_index()

# Create timestamp column using year, month, and day columns
df_combined_grp['timestamp'] = pd.to_datetime(df_combined_grp[['year', 'month', 'day']])

# Add geometry and center latlon columns
df_combined_grp = pd.merge(df_combined_grp, df_combined2[['bboxid', 'geometry_bbox', 'center_latlon']].drop_duplicates(), on='bboxid', how='left')
df_combined_grp.head(3)

,bboxid,year,month,day,ground_truth,timestamp,geometry_bbox,center_latlon
0,bbox_10_14,2016,11,9,0.0,2016-11-09,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"
1,bbox_10_14,2016,11,10,0.0,2016-11-10,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"
2,bbox_10_14,2023,11,3,1.0,2023-11-03,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)"


In [17]:
# Create a complete date range for each bboxid (2016-2024)
full_date_range = pd.date_range(start='2016-01-01', end='2024-05-31')

# Function to reindex each group by full monthly date range
def reindex_group(group):
    
    # Reindex the group based on the full date range
    group = group.set_index('timestamp').reindex(full_date_range).reset_index()
    
    # Rename the index column back to 'date'
    group = group.rename(columns={'index': 'timestamp'})
    
    # Fill the group_id back in for the missing dates
    group['bboxid'] = group['bboxid'].fillna(method='ffill').fillna(method='bfill')
    
    return group

# Separate the geometric columns
geom_cols = ['bboxid', 'geometry_bbox', 'center_latlon']
df_temp = df_combined_grp[geom_cols].copy()
df_temp = df_temp.drop_duplicates(subset=['bboxid'])
df_combined_grp_wo_gem_cols = df_combined_grp.drop(['geometry_bbox','center_latlon'], axis=1)

# Apply the reindexing function to each group
df_combined_grp_wo_gem_cols = df_combined_grp_wo_gem_cols.groupby('bboxid').apply(reindex_group).reset_index(drop=True)

# Merge the geometric columns back into the reindexed DataFrame
df_combined_grp = pd.merge(df_combined_grp_wo_gem_cols, df_temp, on='bboxid', how='left')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/3067946300.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['bboxid'] = group['bboxid'].fillna(method='ffill').fillna(method='bfill')
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/3067946300.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['bboxid'] = group['bboxid'].fillna(method='ffill').fillna(method='bfill')
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/3067946300.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['bboxid'] = group['bboxid'].fillna(method='ffill').fillna(method='bfill')
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/3067946300.py:14: FutureWarning: Series.f

In [19]:
df_combined_grp.shape

(13580932, 8)

In [20]:
# Load weather data
df_temper = pd.read_csv('data/df_temperature_sf.csv')

# Convert date column to datetime
df_temper['timestamp'] = pd.to_datetime(df_temper['date'])

# Select required columns
required_cols = ['timestamp','max','min','precipitation']
df_temper = df_temper[required_cols]

# Merge the weather data with the main dataframe
df_combined_grp = df_combined_grp.merge(df_temper, on='timestamp', how='left')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/1215225631.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_temper['timestamp'] = pd.to_datetime(df_temper['date'])


In [21]:
# Drop geometry_bbox and center_latlon columns
df_combined_grp = df_combined_grp.drop(['geometry_bbox', 'center_latlon'], axis=1)

# Add bboxID and GEOID from bbox_coord data frame
df_combined_grp = df_combined_grp.merge(bbox_coord[['bboxid', 'GEOID', 'geometry_bbox', 'center_latlon']], on='bboxid', how='left')

In [22]:
# Create a new data frame
df = df_combined_grp.copy()

# Load ACS data (2020)
acs_2020 = pd.read_csv('data/acs_data_5y_sf_2020.csv')

# Create columns for race ratio
acs_2020['white_ratio'] = acs_2020['race_white'] / acs_2020['total_population']
acs_2020['black_ratio'] = acs_2020['race_black'] / acs_2020['total_population']
acs_2020['year'] = 2020

# Create a GEOID column
acs_2020['state'] = acs_2020['state'].astype(str)
acs_2020['county'] = acs_2020['county'].astype(str)
acs_2020['tract'] = acs_2020['tract'].astype(str)
acs_2020['block group'] = acs_2020['block group'].astype(str)
acs_2020['state'] = acs_2020['state'].apply(lambda x: x.zfill(2))
acs_2020['county'] = acs_2020['county'].apply(lambda x: x.zfill(3))
acs_2020['tract'] = acs_2020['tract'].apply(lambda x: x.zfill(6))
acs_2020['GEOID'] = acs_2020['state'] + acs_2020['county'] + acs_2020['tract'] + acs_2020['block group']

# Select required columns
acs_2020 = acs_2020[['GEOID','total_population','white_ratio','black_ratio','hh_median_income','year']]

# Load ACS data (2022)
acs_2022 = pd.read_csv('data/acs_data_5y_sf_2022.csv')

# Create columns for race ratio
acs_2022['white_ratio'] = acs_2022['race_white'] / acs_2022['total_population']
acs_2022['black_ratio'] = acs_2022['race_black'] / acs_2022['total_population']
acs_2022['year'] = 2022

# Create a GEOID column
acs_2022['state'] = acs_2022['state'].astype(str)
acs_2022['county'] = acs_2022['county'].astype(str)
acs_2022['tract'] = acs_2022['tract'].astype(str)
acs_2022['block group'] = acs_2022['block group'].astype(str)
acs_2022['state'] = acs_2022['state'].apply(lambda x: x.zfill(2))
acs_2022['county'] = acs_2022['county'].apply(lambda x: x.zfill(3))
acs_2022['tract'] = acs_2022['tract'].apply(lambda x: x.zfill(6))
acs_2022['GEOID'] = acs_2022['state'] + acs_2022['county'] + acs_2022['tract'] + acs_2022['block group']

# Select required columns
acs_2022 = acs_2022[['GEOID','total_population','white_ratio','black_ratio','hh_median_income','year']]

# Separate df into prior 2021 and after 2021
df_prior_2021 = df[df['timestamp'] < '2021-01-01']
df_after_2021 = df[df['timestamp'] >= '2021-01-01']

# Merge ACS data with the main dataframe (by ACS year)
df_prior_2021 = df_prior_2021.merge(acs_2020, on='GEOID', how='left')
df_after_2021 = df_after_2021.merge(acs_2022, on='GEOID', how='left')
df = pd.concat([df_prior_2021, df_after_2021])

# Select required columns
df = df[['timestamp','bboxid','GEOID','geometry_bbox', 'center_latlon','max','min','precipitation','total_population','white_ratio','black_ratio','hh_median_income','ground_truth']]

In [25]:
df

,timestamp,bboxid,GEOID,geometry_bbox,center_latlon,max,min,precipitation,total_population,white_ratio,black_ratio,hh_median_income,ground_truth
0,2016-01-01,bbox_10_14,060750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",50.0,40.0,0.00,506,0.806324,0.00000,53438,NaN
1,2016-01-02,bbox_10_14,060750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",51.0,45.0,0.00,506,0.806324,0.00000,53438,NaN
2,2016-01-03,bbox_10_14,060750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",55.0,46.0,0.02,506,0.806324,0.00000,53438,NaN
3,2016-01-04,bbox_10_14,060750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",57.0,47.0,0.09,506,0.806324,0.00000,53438,NaN
4,2016-01-05,bbox_10_14,060750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",54.0,50.0,1.18,506,0.806324,0.00000,53438,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5509241,2024-05-27,bbox_9_68,060750234002,POLYGON ((-122.38767944786734 37.7172190397561...,"(37.71793953055515, -122.38676050605784)",64.0,52.0,0.00,1517,0.187871,0.13909,29442,NaN
5509242,2024-05-28,bbox_9_68,060750234002,POLYGON ((-122.38767944786734 37.7172190397561...,"(37.71793953055515, -122.38676050605784)",67.0,54.0,0.00,1517,0.187871,0.13909,29442,NaN
5509243,2024-05-29,bbox_9_68,060750234002,POLYGON ((-122.38767944786734 37.7172190397561...,"(37.71793953055515, -122.38676050605784)",76.0,53.0,0.00,1517,0.187871,0.13909,29442,NaN
5509244,2024-05-30,bbox_9_68,060750234002,POLYGON ((-122.38767944786734 37.7172190397561...,"(37.71793953055515, -122.38676050605784)",74.0,54.0,0.00,1517,0.187871,0.13909,29442,NaN


In [26]:
df['ground_truth'].value_counts()

ground_truth
1.0     178832
0.0      44852
2.0      33816
3.0       9903
4.0       3442
5.0       1252
6.0        524
7.0        233
8.0         96
9.0         50
10.0        26
11.0         9
12.0         7
13.0         5
15.0         2
Name: count, dtype: int64

In [27]:
# Remove GEOIDs with zero population (3 GEOIDs)
df = df[df['total_population'] > 0]

# Replace negative values with the mean
df['hh_median_income'] = df['hh_median_income'].replace(-666666666, np.nan)
df['hh_median_income'] = df['hh_median_income'].fillna(df['hh_median_income'].mean())

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/1525337861.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hh_median_income'] = df['hh_median_income'].replace(-666666666, np.nan)
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_94087/1525337861.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hh_median_income'] = df['hh_median_income'].fillna(df['hh_median_income'].mean())


In [28]:
# Save the cleaned dataframe
df.to_csv('data/df_cleaned_20250617.csv', index=False)